<a href="https://colab.research.google.com/github/icculp/route-66-scrape_public/blob/master/places.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import json

In [1]:
places = {'Allen’s Conoco Fillin’ Station': {'address': '101 S Main St\nCommerce, OK 74339', 'description': 'Allen’s Conoco Fillin’ Station in Commerce was built in 1929 or 1930 by F.D. Mitchell. It became a Phillips 66 Gas Station in 1938.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/f1c9dbe87566a044827c7ab2439271fa_.jpeg'}, 'Mickey Mantle’s Childhood Home': {'address': '319 S Quincy St\nCommerce, OK 74339', 'description': 'Private residence', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Mickey Mantle Field': {'address': '400 S Mickey Mantle Blvd\nCommerce, OK 74339', 'description': 'In 2010, Commerce officials dedicated a statue of Mickey Mantle designed by local artist Nick Calcagno. The statue stands beyond center field at the Commerce High School baseball field where Mantle first began playing baseball.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/d2c906eedda0e9039145925ed6f224d2_.jpeg'}, 'Picher – Ghost Town': {'address': 'Picher, OK 74360', 'description': "In 1913, as the Tri-State district expanded, lead and zinc ore were discovered on Harry Crawfish's claim and mining began. A town site developed overnight around the new workings and was named Picher in honor of O. S. Picher, owner of Picher Lead Company. The city was incorporated in 1918, and by 1920, Picher had a population of 9,726. Peak population occurred in 1926 with 14,252 residents. Mining ceased in 1967. The contaminated water from some 14,000 abandoned mine shafts, 70 million tons of mine tailings, and 36 million tons of mill sand and sludge remained as a huge environmental cleanup problem. As a result of national legislation to identify and remediate such environmentally hazardous sites, in 1980 the area was designated as part of the Tar Creek Superfund site.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/8e9001ceffee2dd23f940079d579a060_.jpeg'}, 'Constable Campbell Memorial': {'address': '618 Commerce Ave\nCommerce, OK 74339', 'description': 'Constable William Campbell was shot and killed by the notorious outlaw gang led by Bonnie and Clyde. The Chief of Police was also shot and wounded in the incident.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'The Hitch n Post Flea Market': {'address': '300-336 Commerce St\nCommerce, OK 74339', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/f5ab0dc1df3120d52f07287a5d01ffb1_.jpeg'}, 'Dairy King': {'address': '100 N Main St\nCommerce, OK 74339', 'description': 'The building originally housed a circa 1930 cottage-style gas station. The old fashioned drive in is known for burgers, soft-serve ice cream and handmade Route 66 cookies shaped like the Route 66 shield. Owners Charles Duboise and his mother Treva serve food with a smile and share the history of the city. NOTE: cash is required as they don’t accept checks or credit cards.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/784b2a73dc3b61249a4ed8953367b112_.jpeg'}, 'Coleman Theater': {'address': '103 N Main St\nMiami, OK 74354', 'description': 'The Coleman Theater opened on April 18, 1929. Counted among the many things that make this theater is unique is the Wurlitzer organ built in 1928 and delivered for installation on Feb. 21, 1929. It sold for $35,000 and is the only Wurlitzer organ built in 1928 that is in its original location. Among the many stars who have performed on the Coleman Theater stage are Will Rogers, Tom Mix, Sally Rand, Bing Crosby, Jim Thorpe, the magician Harry Blackstone and Moscelyn Larkin.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/b24ca5ee42d2465dbd25b56dcfe47dfd_.jpeg'}, 'Miami Convention and Visitors Bureau': {'address': '101 North Main Street\nMiami, OK 74354', 'description': 'ROUTE 66 PASSPORT STAMP PLACE The Route 66 Visitor Center is a great place to stop for free brochures as you enter Oklahoma in Northeast Oklahoma. The gift shop offers one of a kind souvenirs to take back home. The center houses the CVB staff that can help you plan your trip across Oklahoma. Find more information online at www.visitmiamiok.com or download the Visit Miami OK app.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/a28359482cf05263a5a9672d55eed1be_.jpeg'}, 'Route 66 Gateway Sign': {'address': '218 S Main St\nMiami, OK 74354', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/39dc0abbaf08b63af52633b28d606acd_.jpeg'}, 'Waylan’s KuKu Hamburgers': {'address': '915 N Main St\nMiami, OK 74354', 'description': 'This restaurant offers traditional American diner far with a twist. Buffalo burgers are also on the menu. This is the last remaining restaurant in what was a regional chain. The sign is original and dates to 1965', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/015682d83d3e6060799c2f88ecd45fc5_.jpeg'}, 'Route 66 Vintage Iron': {'address': '128 S Main St\nMiami, OK 74354', 'description': 'Route 66 Vintage Iron, a motorcycle museum and gift shop, opened in May of 2006.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/792cdb1b70be7d6ea943648244caf95f_.jpeg'}, 'Rte 66 Historic Marker': {'address': 'E 140 Rd\nMiami, OK 74354', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/aacdcee106e34bdec30bab4caf64ea8d_.jpeg'}, 'Nowhere on Route 66': {'address': '300 SE 1st St\nAfton, OK 74331', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/98ade581fb3c42766b072a0ffb35d604_.jpeg'}, 'Rest Haven Sign': {'address': '108 US-69\nAfton, OK 74331', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/aa83e6fc0cc5f0d6eae4191fe65fda7d_.jpeg'}, 'Afton Station': {'address': '12 SE 1st St\nAfton, OK 74331', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/4e9284405a40a86aa4b5b9b99ab74fcc_.jpeg'}, 'Vinita Bridge': {'address': 'US69\nVinita, OK 74301', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0dc779b74689f10fbad42eb8402131a4_.jpeg'}, 'Clanton’s Cafe': {'address': '319 E Illinois Ave\nVinita, OK 74301', 'description': 'Clanton’s has been a family tradition in Vinita Oklahoma since 1927. It is the oldest continually owned family restaurant on Route 66 in the state of Oklahoma.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'The Coffee Shelter': {'address': '121 S Wilson St\nVinita, OK 74301', 'description': 'European Coffee', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/e56e4bb42e6d6f46ab35c7b65c64ad35_.jpeg'}, 'Old Service Station': {'address': 'OK-8\nHinton, OK', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Route 66 Inn': {'address': '519 S Wilson St\nVinita, OK 74301', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/d6873455946357730fedd2ea504b720e_.jpeg'}, 'Diamond Rio': {'address': '437950 US-60\nVinita, OK 74301', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/340de1d2aba19238f1c8b9d64475037a_.jpeg'}, 'Hi-Way Café': {'address': '437918 US-60\nVinita, OK 74301', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/d0a2965e3052f75d2062d0d7fa7d72d9_.jpeg'}, 'Pryor Creek Bridge': {'address': '58 S4260 Rd\nChelsea, OK 74016', 'description': 'This bridge was built in 1926', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0b5fba15031d32324580c3775af3b95f_.jpeg'}, 'Pedestrian Underpass': {'address': 'Chelsea, OK 74016', 'description': 'Built in the 1930s to facilitate safe pedestrian crossing of Route 66. In 2016, the City of Chelsea gave the pedestrian underpass a cleaning and commissioned artist Kenneth Hollingshead to start creating a historical Route 66 Pedestrian Underpass Mural. This mural pays tribute to historical elements of Chelsea, from a 1930s gas station to the Hogue House.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2b953624098acb102dd180b19f8bcfe5_.jpeg'}, 'Ed Galloway’s Totem Pole Park': {'address': '21300 OK-28 A\nChelsea, OK 74016', 'description': "Ed Galloway's Totem Pole Park is a folk art masterpiece consists of eleven elaborately decorated concrete tee pee towers and one ornate building on 14 acres. The park is located 3.5 miles east of historic U.S. Route 66 and Foyil. It was added to the National Register of Historic Places on March 30, 1999", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/378a583c45f7c3f1cf0630eeb8519140_.jpeg'}, 'Bunion Derby Historic Marker': {'address': '12564 Andy Payne Blvd\nFoyil, OK 74017', 'description': 'Commemorates Andy Payne who won the 1928 Bunion Derby. Andy Payne was a native of Foyil and the original alignment of Route 66 is signed Andy Payne Boulevard in that town.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, '1920s Texaco station': {'address': 'Andy Payne Blvd\nFoyil, OK 74017', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Claremore Motor Inn': {'address': '1709 N Lynn Riggs Blvd\nClaremore, OK 74017', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/be9a3df3d025abfaa025794d2da4c2ca_.jpeg'}, 'J.M. Davis Museum': {'address': '330 N J M Davis Blvd\nClaremore, OK 74017', 'description': "The world's largest collection of historic firearms as well as displays of antique German steins, John Wayne exhibits, 19th century music boxes, and Cherokee cultural exhibits.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/159ebaa2da6e404143b52e961769d818_.jpeg'}, 'Will Rogers Museum': {'address': '1720 W Will Rogers Blvd\nClaremore, OK 74017', 'description': "The Will Rogers Memorial Museum is a 19,052-square-foot museum in Claremore, Oklahoma that memorializes entertainer Will Rogers. The museum houses artifacts, memorabilia, photographs, and manuscripts pertaining to Rogers' life, and documentaries, speeches, and movies starring Rogers are shown in a theater.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/8a3444c057ca2a0c64296a6df355668f_.jpeg'}, 'Ron’s Hamburgers & Chili': {'address': '1220 S Lynn Riggs Blvd\nClaremore, OK 74017', 'description': 'Established in 1975, this is an outpost of a long-running chain known for its old-fashioned burgers, chili & draft root beer.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Chelsea Motor Inn': {'address': '325 E Layton St\nChelsea, OK 74016', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/75d61274499d8f409709695de55af29f_.jpeg'}, 'Chelsea Motel': {'address': '100 E 1st St\nChelsea, OK 74016', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/c2861591bb98b77192d9a7a3753caafd_.jpeg'}, 'Tulsa Fire Station 66 “Keepers of the Mother Road”': {'address': '14333 E 11th St\nTulsa, OK 74108', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/df97c5354ebbd82d8ead216d3faf11e2_.jpeg'}, 'Twin Bridges Verdigris': {'address': 'Verdigris, OK 74019', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/906e807e999a0f93f9bc47020ad0dd1c_.jpeg'}, 'Arrowood Trading Post': {'address': 'Rt 66\nCatoosa, OK 74015', 'description': 'The Arrowood Trading Post opened in 1990. The building dates to 1952. The building formerly housed the “Chief Wolf Robe Hunt Trading Post” also known as “Catoosa Trading Post.” Wolf Robe Hunt, an Acoma Indian, sold beautiful Osage beadwork, Navajo jewelry, and Acoma pottery (supplied by relatives) at his store in Tulsa. In 1952, Hunt moved his business to Catoosa across the street from sister-in-law, Zelta Davis’s Alligator Farm and Blue Whale.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2c9d0ccba8ffcfe2336d510e8e150755_.jpeg'}, 'Blue Whale': {'address': '2600 OK-66\nCatoosa, OK 74015', 'description': 'Built between 1970 and 1972 as part of a recreational complex by Hugh Davis. The whale, pond, park and a small zoo served as a popular recreational point for the community and Route 66 travelers for two decades. After a decade of abandonment, in 2010 it was refurbished', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/b69599544e0233041bbfdf8ed6b20ff4_.jpeg'}, 'Catoosa Historical Museum': {'address': '217 S Cherokee St\nCatoosa, OK 74015', 'description': "Housed in the city's historic depot this museum highlights Catoosa's colorful history through displays including vintage photographs of Catoosa's early residents, a mailbox that was used when mail was delivered on horseback, a train depot log book dating from 1897 and a check from the Farmers and Merchants Bank dated 1907.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/69d4549faa2c4232d67ed914dc5927ea_.jpeg'}, 'Correll Museum': {'address': '19934 E Pine St\nCatoosa, OK 74015', 'description': 'A leading institution of the Route 66 Archives and Research Collaboration, and is a key contact for Route 66 archival research in Oklahoma.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/3437cb5842c5a139f5eede20eccc3cd8_.jpeg'}, 'Catoosa Inn & Suites': {'address': '40 S 193rd E Ave\nTulsa, OK 74108', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/3772e3b13a9463d248bd8f064a9ca281_.jpeg'}, 'Tulsa Sign Route 66': {'address': '4207-4275 Southwest Blvd\nTulsa, OK 74107', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'The Campbell Hotel': {'address': '2636 E 11th St\nTulsa, OK 74104', 'description': 'The Campbell Hotel & Event Centers is a luxury hotel and event center consisting of 26 uniquely decorated rooms, a bar & lounge, and a full-service spa & salon. Originally built in 1927, and now listed on the National Register of Historic Places, the Casa Loma Hotel was housed inside the Max Campbell Building. The hotel was conveniently located at the end of the trolley line that ran from downtown Tulsa to Delaware Avenue on 11th Street. Businesses housed on the hotel’s street level included a barbershop, beauty shop, Crawford Drug Store, and Safeway grocery store.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Golden Driller Statue': {'address': 'Tulsa Expo Center, 4145 E 21st St\nTulsa, OK 74114', 'description': 'The Golden Driller is a 75-foot-tall, 43,500-pound statue of an oil worker, in Tulsa, Oklahoma. It was built from a steel frame, covered with concrete and plaster. It is the sixth-tallest statue in the United States and has been located in front of the Tulsa Expo Center since 1966.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, '918 Coffee': {'address': '2446 E 11th St\nTulsa, OK 74104', 'description': 'European Coffee', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/1aa55311b423aa2d15445996ec0e31cb_.jpeg'}, 'Mother Road Market': {'address': '1124 S Lewis Ave\nTulsa, OK 74104', 'description': "ROUTE 66 PASSPORT STAMP PLACE Mother Road Market is Tulsa's first food hall, where you can eat and shop at over 20 local businesses under one roof! Mother Road Market also features a full bar (including coffee drinks!), nine holes of Route 66-themed mini golf, three murals by local artists, children's play areas and more! Shop Route 66-themed and Tulsa-themed souvenirs, t-shirts, hats and more. Travelers with the Route 66 passport will receive a free Mother Road Market tote that can be redeemed inside at the Kitchen 66 General Store.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/fd7193b2e31ddedb98d271c9d3f9f776_.jpeg'}, 'Buck Atom’s Cosmic Curios on 66': {'address': '1347 E 11th St\nTulsa, OK 74120', 'description': "A fun, quirky souvenir shop with an array of memorable photo ops. on site is the Airbnb Buck's Cosmic Crash Pad. Located right next door to the store is a fresh way to experience Historic Route 66, shop for local art + souvenirs, and enjoy the great surroundings and attractions of Tulsa.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/73212edea9db2fbc134064bd96bd3ea4_.jpeg'}, 'Brookshire Motel': {'address': 'S Garnett Rd&E 11th St S\nTulsa, OK 74128', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/618b69425970acf0edcc805cd94e6a65_.jpeg'}, 'Oasis Motel': {'address': '9303 E 11th St\nTulsa, OK 74112', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/f4fa71609c9d0ed2c52532e322dc66a8_.jpeg'}, 'Rose Bowl': {'address': '7419 E 11th St\nTulsa, OK 74112', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/dbff3bdccf6b4348547021941f42bea4_.jpeg'}, 'Foolish Things Coffee Company': {'address': '1001 S Main St\nTulsa, OK 74119', 'description': 'European Coffee', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/9ac2d3421f28c3c23ee8120b14a8ff1d_.jpeg'}, 'The Blue Dome': {'address': '202 S Elgin Ave\nTulsa, OK 74120', 'description': 'Built by T.J. Chastain in 1925, the station was the crown jewel of the Tulsa based Chastain Oil Company. It was designed with an apartment for the manager upstairs. The station was one of the first in Oklahoma to offer a car wash and free pressurized air.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2558f8d7aee1a921e682b84e4c3ee23d_.jpeg'}, 'Meadow Gold Sign': {'address': '1324 E 11th St\nTulsa, OK 74120', 'description': 'This sign dates to he 1930s and was originally mounted on the roof of the Beatrice Foods building located at Eleventh and Lewis along Route 66. Scheduled for demolition in 2004, a coalition that included the National Park Service, City of Tulsa, Tulsa Foundation for Architecture and Oklahoma Route 66 Association funded restoration of the sign, the building of the kiosk, and installation.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/e9786655d93928abc59f973bbb321e0e_.jpeg'}, 'Soul City Gastropub': {'address': '1621 E 11th St\nTulsa, OK 74120', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/cbb8a4a6f4fae1a5640ec16f6617a47f_.jpeg'}, 'Tally’s Good Food Cafe': {'address': '1102 S Yale Ave\nTulsa, OK 74112', 'description': 'A classic Route 66 diner with lots of neon, a great menu and friendly service.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/6512550818923ee8b10b39e2ce0e98b4_.jpeg'}, 'Circle Inn': {'address': '8945 E Admiral Pl\nTulsa, OK 74115', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/5ffcce982bf739feff5b45f79ca1cd6f_.jpeg'}, 'Hank’s Hamburgers': {'address': '8933 E Admiral Pl\nTulsa, OK 74115', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2535d6d42aa5d4b602dbf8ea28fb1762_.jpeg'}, 'Thelmas Bar Sign': {'address': '2008 E Admiral Blvd\nTulsa, OK', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/e30f062aab6539368d2d6d306fd365b2_.jpeg'}, 'Route 66 Overpass': {'address': 'Southwest Blvd\nTulsa, OK 74128', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/bcac90d1a4d69672d5e08e1d29470717_.jpeg'}, 'Cyrus Avery Centennial Plaza': {'address': 'Southwest Blvd\nTulsa, OK 74127', 'description': 'The plaza is a commemoration to Cyrus Avery, the man deemed the Father of Route 66. The centerpiece is the East Meets West sculpture that depicts Avery in a Model T encountering a horse drawn wagon.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/83b4913cb0dde29200c8199a5b6a8c7d_.jpeg'}, 'Cyrus Avery Memorial Bridge': {'address': 'Southwest Blvd\nTulsa, OK 74128', 'description': 'Built by the Missouri Valley Bridge & Iron Company in 1916, 1917 at a cost of $180,000, the bridge originally supported a railroad line, two lanes of vehicular traffic and pedestrian sidewalks on both sides. In 1929 it was transformed into a highway bridge to meet the increasing needs of traffic on Route 66. Five years later a major upgrade transformed it into a 52 foot wide bridge deck to accommodate four lanes of traffic. It remained in use until 1980.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/129cd76d66ea604f6c4aa8b520fd5236_.jpeg'}, 'Howard Park': {'address': '2501 Southwest Blvd\nTulsa, OK 74107', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/bbd848d75416d307b1a705ca7c964e35_.jpeg'}, 'Goodwill Sign': {'address': '3100-3248 Southwest Blvd\nTulsa, OK 74107', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/cbb509875c6e4422f8785d43e4b07eeb_.jpeg'}, 'Route 66 Historical Village': {'address': '3770 Southwest Blvd\nTulsa, OK 74107', 'description': 'Children\'s amusement center with photo ops including a tourism information center modeled after a 1930s gas station, steam locomotive, and oil derrick. The oil derrick stands 194 feet high and is on the historic site of the first oil strike on June 25, 1901 that made Tulsa the "Oil Capital of the World."', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/bccff47e652afff23f55126dcb5c6ec1_.jpeg'}, 'Ollies Station Restaurant': {'address': '4070 Southwest Blvd\nTulsa, OK 74107', 'description': 'Modest mainstay with a railroad motif serving homestyle American comfort foods, plus a lunch buffet.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'The Lunchbox Cafe / La Lonchera': {'address': '4261 Southwest Blvd\nTulsa, OK 74107', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/b3fc51388922b63eee2e0a90c9336510_.jpeg'}, 'Junction 66': {'address': '309 N Mission St,\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/9a60a3b69e293453b12480b8bcd109ac_.jpeg'}, 'TeePee Drive-In Theater': {'address': 'W Ozark Trail\nSapulpa, OK 74066', 'description': 'The goal is to buy, retool this location. it shall be a family gathering location for all to enjoy while, experinceing a past gone by. Donate please at https://www.gofundme.com/5ku47zc', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/344a5632055f1cdd68f8f2848e08e272_.jpeg'}, 'Meltdown Diner': {'address': '4377 Southwest Blvd\nTulsa, OK 74107', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/4ac94da050eb66c3a531ccbccef33152_.jpeg'}, 'Happy Burger': {'address': '215 N Mission St\nSapulpa, OK 74066', 'description': 'Classic and gourmet burgers including peanut butter bacon cheeseburger, fried pickles and chili cheese fries', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Sapulpa Historical Society & Fire Museum': {'address': '124 E Lee Ave\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/6fc6b3ba8a59937cecf68af562f850c0_.jpeg'}, 'Benson Lumber Co.': {'address': '114 E Lee Ave\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/7bb1e34fa9896bcc9aa200ed4e2982ab_.jpeg'}, 'Sapulpa Museum': {'address': '100 E Lee Ave\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/49f1c25e410e052e9dd220be53308403_.jpeg'}, 'Waite Phillips-Barnsdall Filling Station Museum': {'address': '26 E Lee Ave\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/7dc58937196b1ac8b224379f6586499d_.jpeg'}, 'Heart of Route 66 Auto Museum': {'address': '13 Sahoma Lake Rd\nSapulpa, OK 74066', 'description': "Home of the world's tallest gas pump this museum maintains a rotating exhibit of historic, vintage, classic and hot rod vehicles.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/3e4b9fda26f4854b6683ef4c086daaaf_.jpeg'}, '1925 Railroad Underpass': {'address': '16375 W Ozark Trail\nSapulpa, OK 74066', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0d30c28f25bb1f57e58e3c5bbda83e74_.jpeg'}, 'Old Bridge': {'address': 'Old Route 66\nBristow, OK 74010', 'description': 'The old bridge was removed from the site in 2018 year.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Plymouth sign tower': {'address': '210 E 9th Ave\nBristow, OK 74010', 'description': 'The sign stands a few feet from the Art Moderne building that was built for the Beard Motor Company which later became Bristow Chrysler Plymouth. The dealership also sold GMC trucks at some point. The business was owned by founding partners Hubert "Red" Beard and Wat Henry. Beard was apparently the public face of the company with Henry being more of a silent partner. The building was constructed in 1945 and early 1946 and the sign was built in 1948 or 1949.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Train Depot and Museum': {'address': '1 Burlington Northern Railroad\nBristow, OK 74010', 'description': 'The Bristow Historical Society is dedicated to the preservation and promotion of the history of Bristow, Oklahoma and the immediate surrounding area.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/b746d2c1a92b76e19dd349eb108d8ccb_.jpeg'}, 'Depew Loop': {'address': 'Main St\nDepew, Oklahoma 74028', 'description': 'From 1926 to 1928, Route 66 looped through Depew. This was the first community bypassed by the highways realignment. Numerous historic buildings of interesting including vintage service stations, hotel ruins and a small café.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Rock Cafe': {'address': '114 W Main St\nStroud, OK 74079', 'description': 'Listed on the National Register of Historic Places in 2001, the Rock Cafe was opened in August 1939 by Roy Rives. The following it year it became an official Greyhound Bus stop and in 1947, resultant of traffic on Route 66, was open 24 hours a day.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/1e7cbb59ca04314d251b88c857e6af2c_.jpeg'}, 'Ed Smalley Centennial park': {'address': '114 W Main St\nStroud, OK 74079', 'description': 'A monument to the Sac and Fox Indian agency established 1869 is located in the park.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/fadda15a31a9dd76d64abec852133452_.jpeg'}, 'Stroud Trading Building': {'address': '201 W Main St\nStroud, OK 74079', 'description': 'Constructed in 1901, this is a two-story red brick building with sandstone trim. At the peak of the front facade the name Stroud Trading Company is built into the wall. At its building, owners boasted that it was the longest floor space of any commercial structure in the Oklahoma Territory. The original owners, the Stroud Trading Company dedicated the second floor of the building as a lodge and operahouse in 1902. This was to respond to the growing interest in cultural things in the Territory. The operahouse operated here until 1925, when the area was remodeled and became the Liberty Hotel. The interior of the first floor has been completely remodeled. In 1917, the wooden floor was replaced with a concrete one. Over the years, the building has been occupied as a garage, a motor company, a grocery, an oil and gas supply company, hotel and restaurant.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/96f67a5bf1e645cd3aedd547b1522b66_.jpeg'}, 'Skyliner Motel': {'address': '717 W Main St\nStroud, OK 74079', 'description': 'The historic Skyliner Motel offers 10 very clean and quiet guest rooms with heat/air and a mini refrigerator. This is a family-owned motel is located within walking distance of the Rock Cafe.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/3fd9deb1fe4c8b028a3ee99c05f08f6a_.jpeg'}, 'Lincoln Motel': {'address': '740 E 1st St\nChandler, OK 74834', 'description': 'Dating to 1939 the historic signage added in the late 1950s has been refurbished.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/7b7fd68a66b22720a51114191bdcd120_.jpeg'}, 'Route 66 Interpretive Center': {'address': '400 E 1st St\nChandler, OK 74834', 'description': 'Built as the Chandler Armory between 1935 and 1937 the facility included offices, a vehicle repair garage, ammunition vault, classrooms, an auditorium and locker rooms. It was he headquarters for the Battery F, Second Battalion of the 160th Field Artillery of the Oklahoma National Guard 45th Infantry Division that served in North Africa, Sicily and Italy during WWII. The hand laid floor in the drill hall was made from more than 156,000 wooden blocks.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/99d06acccea448bbd904eeba01cc329e_.jpeg'}, 'McJerry’s Route 66 Gallery': {'address': '306 Manvel Ave\nChandler, OK 74834', 'description': 'The gallery for internationally acclaimed artist Jerry McClanahan, creator of the EZ 66 Guide For Travelers.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/068e36df635db228f7b371a15096983d_.jpeg'}, 'Phillips 66 Station': {'address': '701 Manvel Ave\nChandler, OK', 'description': 'An historic market on the site reads, "This former Phillips 66 station was built in 1930 and operated as a gas station until 1992. The garage at the left was a late 1950s addition. In 1967, it became a Skelly station. In 1983, it operated as a Getty station. After that, it was a Texaco. The station was restored in 1999.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/88b62b7c91c2575096c8551b1199ff3d_.jpeg'}, 'Service Station': {'address': '709 Manvel Ave\nChandler, OK', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Lincoln County Museum of Pioneer History': {'address': '719 Manvel Ave\nChandler, OK 74834', 'description': 'Founded on April 26, 1954 the collections within the museum include county wide photographs and artifacts from families dating back to before the land run of 1891.The Genealogy and Historical Research departments of the museum boosts microfilm and hard files of events, businesses, families, military and general history of the entire county going back to the Spanish American War of 1898', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/4145105ce9bcfc6e59880820453313d1_.jpeg'}, 'Boomarang Diner': {'address': '912 Manvel Ave\nChandler, OK 74834', 'description': "'50s-style diner chain for American eats such as burgers & chicken-fried steak, plus breakfast fare.", 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Pops Arcadia': {'address': '660 OK-66\nArcadia, OK 73007', 'description': 'A quirky blend of classic malt shop and futuristic convenience store. The claim to fame is hundreds of different kind of soda pop.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2d6883444e8e2ec725a090fdfd050e0d_.jpeg'}, 'Firefighters Museum': {'address': '2716 NE 50th St\nOklahoma City, OK 73111', 'description': 'The Oklahoma State Firefighters Museum is a museum owned and administered by the Oklahoma State Firefighters Association.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Paul McCartney Sign': {'address': 'Old Hwy 66\nArcadia, OK 73007', 'description': 'Paul McCartney stopped here on August 5th, 2008, with his future wife, Nancy Shevell, during a cross-country trip in a 1989 Ford Bronco on the Mother Road shortly after his 66th birthday.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0709613e5cec03b0e9bf5df657962467_.jpeg'}, 'Arcadia Round Barn': {'address': '107 OK-66\nArcadia, OK 73007', 'description': "The Round Barn was built by local farmer William Harrison Odor in 1898 using native bur oak boards soaked while green and forced into the curves needed for the walls and roof rafters. The Arcadia Round Barn is 60 feet (18 meters) in diameter and 43 feet (13 meters) tall. In April 1946, ownership of the barn passed to Frank and Katie Vrana. For the next 30 years, Vrana used the barn to store hay and as a work place. In 1977, the barn was placed on the National Register of Historic Places, but it was near the point of collapse. Luke Robison, a retired builder and carpenter, became aware of the barn’s plight and formed The Arcadia Historical and Preservation Society with his wife Anna and Beverly White. On May 27, 1988 Frank Vrana's descendants donated the barn to the Society. Before work could begin the roof collapsed on June 29, 1988. Restoration work continued. Much of the labor was performed by volunteers, mostly retirees like Robison, who called themselves the Over-the-Hill Gang.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0fde0fe74036160034c81175060ce714_.jpeg'}, '45th Infantry Division Museum': {'address': '2145 NE 36th St\nOklahoma City, OK 73111', 'description': 'State-run, free-entry indoor/outdoor museum displaying military history artifacts & memorabilia.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Seaba Station Museum': {'address': '336992 OK-66\nWarwick, OK', 'description': 'Seaba Station was built in 1921. It was built by John and Alice Seaba. It was originally a DX station serving Nevrnox gasoline. John turned the gas station into a machine and engine rebuilding shop. He established a company to rebabit connecting rods. The business was sold to Victor and June Briggs in 1951. Sonny and Sue Preston bought the building in 1995 and after many years of restoration opened Seaba Station Antiques & Gifts. The build was sold in October of 2007 to Gerald Tims and Jerry Ries who refurbished the building and created the motorcycle museum', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/dfa59ab68c594e40286bfb5e6276a76a_.jpeg'}, 'Threatt Filling Station Route 66': {'address': 'Route 66\nLuther, OK 73054', 'description': 'Built around 1915 using local sandstone, the “house type” station has Bungalow/Craftsman features typical of the period. Each of its four gables has wide eaves and triangular braces. The prominent front-facing gable is positioned over wide double-entry doors. That entry used to have spring-loaded screen doors, the kind that banged, bounced, and banged again when people came and went. The original 1915 gasoline pumps had glass containers on top so the attendant could measure how much gas went into the car. In their place now are two 1940s enamel pumps complete with the old geared system to indicate the flow of gas with clicking metal numbers. The signage that used to top the pole between the pumps is gone, but the old lights that once illuminated the front of the station are still in place. Except for a 1961 addition to the rear of the property, the station’s form is virtually unchanged from the way it looked when Allen Threatt built it.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'DJ’S BBQ': {'address': '204 N Ash St\nLuther, OK 73054', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Oklahoma Railway Museum': {'address': '3400 NE Grand Blvd\nOklahoma City, OK 73111', 'description': 'The Oklahoma Railway Museum preserves railroad history while teaching about how the railroads affected national development. In addition to the physical preservation of the rail lines, equipment, and artifacts, the museum tells the stories of the people whose lives were changed by the railroads.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'National Cowboy & Western Heritage Museum': {'address': '1700 NE 63rd St\nOklahoma City, OK 73111', 'description': 'The National Cowboy & Western Heritage Museum is America’s premier institution of Western history, art and culture. Founded in 1955, the Museum, located in Oklahoma City, collects, preserves and exhibits an internationally renowned collection of Western art and artifacts while sponsoring dynamic educational programs to stimulate interest in the enduring legacy of the American West.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/81a01eeb281b80b94d0d8767354ad7d1_.jpeg'}, 'Tower Theatre': {'address': '425 NW 23rd St\nOklahoma City, OK 73103', 'description': 'Built in 1937, this theater now features a stage for live music as well as screen to serve as a movie theater. The unique, restored marquee and signage is a popular photo op.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/720b088adea9d76ac46b54d066292dce_.jpeg'}, 'Owl Court': {'address': '742 W Britton Rd\nOklahoma City, OK 73114', 'description': 'According to the National Park Service, the Owl Court was built to serve travelers on the 1931 Route 66 bypass around Oklahoma City. The Owl Court complex operated a gas station, motel, and cafe until the 1970s', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/902972ca625c271a2a67080b92d1d028_.jpeg'}, 'Oklahoma State Capitol': {'address': '2300 N Lincoln Blvd\nOklahoma City, OK 73105', 'description': 'Built in 1917, the Greco-Roman structure was topped with a dome in 2002 and houses murals and paintings of Oklahoma’s history. The Oklahoma Veterans Memorial is located outside on the north side of the capitol complex. Four walls depict scenes of World War I, World War II, Korea and Vietnam wars, along with a bronze statue of “The Big Guy” who stands eight and a half feet tall on a three and a half foot pedestal.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/23d0976378bf7632cf66bb630614ca27_.jpeg'}, 'The Gold Dome': {'address': '1112 NW 23rd St\nOklahoma City, OK 73106', 'description': 'Built in 1958 this was the fifth building in the world to use the geodesic dome design created by Buckminster Fuller. It was the first building utilize anodized gold aluminum and is listed on the National Register of Historic Places.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/6c45e3d679c3b9b55e631568e83079c2_.jpeg'}, 'Milk Bottle Grocery': {'address': '2426 N Classen Blvd\nOklahoma City, OK 73106', 'description': "The Milk Bottle Grocery, located at 2426 N. Classen Boulevard in Oklahoma City, Oklahoma, is a grocery building with a large metal Braum's milk bottle atop its roof. The store was constructed in 1930, and the milk bottle was added in 1948. The bottle was designed to draw attention to the store, as the tall bottle would be visible to automobile traffic along Classen Boulevard, which was part of U.S. Route 66 at the time; it also served as an advertisement for the dairy industry. The store is also one of the few triangular buildings in Oklahoma City, as it occupies a corner lot in an area where Classen Boulevard cuts diagonally through the city's street grid. Due to its shape, the store was known as the Triangle Grocery from 1940 until 1948, when it became the Milk Bottle Grocery due to its new statue. The building was added to the National Register of Historic Places on March 5, 1998. The Saigon Baguette, a Vietnamese bánh mì shop, occupied the space for several years. In August 2014 it was announced that the building would be restored to its original appearance.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/ffed550a41b2529ce958b5674e065769_.jpeg'}, 'Hoover Service': {'address': 'N Classen Blvd\nOklahoma City, OK 73106', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/95436ab4c446f3c27d89f18136b77d93_.jpeg'}, 'Ranger Motel': {'address': '1201 SE 27th St\nEl Reno, OK 73036', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/99af3fbdd5b5c3ef1736e6b05c65e8ec_.jpeg'}, 'Route 66 Reno sign': {'address': 'Route 66\nEl Reno, OK 73036', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/653b955c3ec9aa4f15e4165bd8f7540e_.jpeg'}, 'Aloha Shave Ice and Coffee Shoppe': {'address': '8000 NW 39th Expy\nBethany, OK 73008', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/1a64b886941213e994dfad9aa1e13f4c_.jpeg'}, 'Lake Overholser Bridge': {'address': '8703-8709 Overholser Dr\nBethany, OK 73008', 'description': 'Listed in the National Register of Historic Places is a unique blending of Parker thru-truss and pony truss design construction. Construction on the bridge commenced in 1924, and opened to traffic in August 1925.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/3f4c925b3c4588b2b40bdbbf950d1da9_.jpeg'}, 'Yukon’s Best Flour Sign': {'address': '301-309 OK-66\nYukon, OK 73099', 'description': 'The elevator with the "Yukon\'s Best Flour" sign is illuminated with individual bulbs rather than neon. The sign was placed on top of the mill in the mid 1940s. The Friends of Yukon\'s Best, Inc. raised funds to refurbish the towering with LED lighting. It was relit in 2013.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/11c1eb070cce3fa926eb63aefe24d98d_.jpeg'}, 'Yukon Mills Sign': {'address': '301-309 OK-66\nYukon, OK 73100', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/f34778e4233e8a2ec1cb962f8c82da34_.jpeg'}, 'Yukon Welcome Sign': {'address': 'W Main St\nYukon, OK 73100', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/aaa34a13835c283b9d49bb74b81a019f_.jpeg'}, 'Yukon’s Best Railroad Museum': {'address': '328-346 Cedar Ave\nYukon, OK 73099', 'description': "The Yukon's Best Railroad Museum is housed in a CRI&P boxcar on West Main Street (Route 66). Double doors were added to the side of the car to ease entry to the museum. The museum features Rock Island Railroad antiques and artifacts, as well as general railroad items as well as a Route 66 exhibit.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/ac800afaf38e675adb52271e40063984_.jpeg'}, 'Historic Fort Reno': {'address': '7107 West Cheyenne Street\nEl Reno, OK 73036', 'description': 'Fort Reno began as a military camp in 1874 in the Indian Wars Era. In 1876 the fort was named by General Phil Sheridan, in honor of his friend Major General Jesse L. Reno, who was killed in the Civil War in 1863 at the Battle of South Mountain in Maryland. The cavalry and infantry stationed at Fort Reno played an important role in the transition of the area from Indian Territory status to Oklahoma statehood in 1907. United States Cavalry units, including the Buffalo Soldiers (Black soldiers of the 9th and 10th U.S. Cavalry, and the 24th and 25th Infantry), and the Cheyenne and Arapaho Indian Scouts, along with the U.S. Marshal Service, maintained the peace on the central plains until the turn of the century. In 1908, Fort Reno became one of three Army Quartermaster Remount Stations for the military, a role which it served through 1948. Specialized horse breeding and training of pack mules became the central focus of activity at Fort Reno. During World War II, 94-acres of an eastern portion of the Fort Reno lands served as an internment work-camp for German and Italian Prisoners of War. They were a part of General Rommel’s Afrikakorp captured in North Africa. in 1944, they built the Post Chapel located to the north of the Parade Grounds. The west side of the historic military cemetery is where seventy German and Italian Prisoners of War are interred.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/14ab76a5bf03016bfcf5d4fa06f0a11f_.jpeg'}, 'Dome House': {'address': 'Rte 66\nHinton, OK 73047', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/14353fae9434a44343150ad3f92b7ab9_.jpeg'}, 'Sid’s Diner': {'address': '300 S Choctaw Ave\nEl Reno, OK 73036', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/0d1258a82f54777f00f93e19da99f690_.jpeg'}, 'Murals of El Reno': {'address': '116 S Choctaw Ave\nEl Reno, OK 73036', 'description': 'One of downtown El Reno’s walls is decorated with a 108-ft. wide mural showing the heritage of El Reno, Canadian County and western Oklahoma across nine panels. Artist Charlie Beecham originally painted this El Reno Heritage Mural in 1980 for OK Federal Savings and Loan Association, and over time, as the paint has worn away, Beecham has volunteered to restore his work with fresh coats of paint. The first panel features the Cheyenne-Araphao reservation, while the second panel represents the Darlington Indian Agency. The third panel highlights Fort Reno, and the fourth panel depicts a Rock Island Railroad train. The fifth panel pays tribute to El Reno as a chartered town in 1889, while the sixth panel details the original Oklahoma Land Run of 1889. In the seventh panel, Beecham painted Caddo Springs Station, while he chose to highlight Jesse Chisholm’s Trail in the eighth panel. In the ninth and final panel, Beecham painted Francisco Coronado traveling through on his 1541 journey through western Oklahoma.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/728098e7be836ea2860035913c8c8b02_.jpeg'}, 'Squawk N Skoot': {'address': 'Sunset Dr\nEl Reno, OK 73036', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/492d53c9cd49554714e47a402dd960ac_.jpeg'}, 'Jobe’s Drive In Sign': {'address': '1220 Sunset Dr\nEl Reno, OK 73036', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/2bcecc0e4c80633cbce2ea8d91658a88_.jpeg'}, 'Muffler Man Art Installation by John Cerney': {'address': 'Route 66\nEl Reno, OK 73036', 'description': 'A California-based artist conceived the cutout paintings of Muffler Men that were erected along Route 66', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/e4a27ca7470272054ea42d56cb0e0d71_.jpeg'}, 'Old Oil Station': {'address': 'US-270\nCalumet, OK 73014', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/7dba1679ea68f5417f7cce35bbf3ce70_.jpeg'}, 'Lucille’s Roadhouse Diner': {'address': '1301 N Airport Rd\nWeatherford, OK 73096', 'description': 'Enduring roadside pit stop with a Route 66-themed, retro diner & upscale steakhouse/cocktail lounge. Also expansive gift shop', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Stafford Air & Space Museum': {'address': '3000 Logan Rd\nWeatherford, OK 73096', 'description': 'The Stafford Air & Space Museum is named for NASA astronaut and Weatherford native Thomas P. Stafford, became a Smithsonian Affiliate in June 2010.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Heartland of America': {'address': '1600 S Frontage Rd\nWeatherford, OK 73096', 'description': 'visitors can see - up close and personal - many artifacts from the late 1800s through the 1950s as they walk through the 32 major historical exhibits and nearly 40 aisle cabinet exhibits. In addition to the main exhibit building, the Heartland Museum features three historic theme buildings on the museum grounds.', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Best Western Plus': {'address': '525 E Main St\nWeatherford, OK 73096', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'The Glancy Motel': {'address': '217 W Gary Blvd\nClinton, OK 73601', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/c261b57ed67ae52338aecf5c7b321c02_.jpeg'}, 'Route 66 Cafe': {'address': '301 W Gary Blvd\nClinton, OK 73601', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/ae3160f30457ef87dde76c79600792c6_.jpeg'}, 'McLain Rogers Park': {'address': 'Jaycee Ln\nClinton, OK 73601', 'description': 'Construction on the McLain Rogers Park, a joint project between the Civil Works Administration and Works Progress Administration, began in 1934. Named for the mayor, the park listed in the National Register of Historic Places in 2004, was accessed via Route 66 and an art deco gateway. The twelve acre park evolved to become a recreational center for area residents as well as a welcome respite for travelers. Amenities included a miniature golf course, tennis courts, a baseball field, picnic area, playground, volleyball court, a bathhouse, and amphitheater.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/141e2975d4104edefe713c0d3cf16903_.jpeg'}, 'Territory Route 66 RV Park & Campgrounds': {'address': '14144 Old 66 Road\nBridgeport, OK 73047', 'description': '30/50 amp service, Full hookups', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/5ca244d279c8a9b1c0e59a61317ea3c2_.jpeg'}, 'Lucille’s Station': {'address': 'Rte 66\nHydro, OK 73048', 'description': "The station with living quarters above is a rare example of rural service stations built before 1930. Carl Ditmore built the facility in 1927 at the intersection of Route 66 and the Provine Road. When added to the National Register of Historic Places in 1997, the original name of Provine Station was used. IN 1941 Carl and Lucille Hamons purchased the property that included a store and motel. Hamon's Court as the property was named at the time. The motel closed in the 1962, Carl died in 1971 and Lucille in 1999.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/314beb9f83103395a6078538797edcf3_.jpeg'}, 'Oklahoma Route 66 Museum': {'address': '2229 W Gary Blvd\nClinton, OK 73601', 'description': 'ROUTE 66 PASSPORT STAMP PLACE For the ultimate Route 66 experience, visit the Oklahoma Route 66 Museum in Clinton OK. The largest history museum from Chicago, Illinois to Santa Monica, California dedicated to the history and culture of Route 66, the most famous highway in the World. The museum’s galleries are designed to offer visitors a personal journey through the history of the nation’s most revered highway. Encounter the iconic ideas, images and myths of the Mother Road. Learn about the dreams and labor needed to make the road a reality. Experience the dust bowl as thousands streamed along the road, away from drought and despair and towards the “land of promise.” Listen to the sounds of the Big Band Era, when the roar of the big trucks and the welcome home cries to returning soldiers dominated the road. Touch the counter and sit in the booth of the 1950’s diner and feel the open road as America’s families vacationed along the length of Route 66. The museum also offers changing special exhibits, focusing on the Rt. 66 experience in our new Now and Future Gallery. And don’t forget to step into our gift shop for some Route 66 Memorabilia!', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/50a85877c98010ef4c9145b754cbc68a_.jpeg'}, 'Tornado Dome': {'address': 'Jaycee Ln\nClinton, OK 73601', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/d19c4a92c60ded84f7ee3b33af2f91de_.jpeg'}, 'Kobel’s Cafe and Gas Station': {'address': 'Historic Route 66\nFoss, OK 73647', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Red Rock Custom': {'address': '21393 Rte 66 N\nFoss, OK 73647', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/89c1a85f6ecb6f5a346b4d861fae0f01_.jpeg'}, 'Cotton Boll Motel': {'address': '605 Old US Hwy 66\nCanute, OK 73626', 'description': 'This motel was established by Woodrow and Viola Peck, cotton farmers, in 1960. The motel closed in 1979', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/f239e88dc45dd08443b884c414f64037_.jpeg'}, 'Washita Motel Sign': {'address': '101-113 Old US Hwy 66\nCanute, OK 73626', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/5c469cd30acd123e489e97394f37ab4a_.jpeg'}, 'The Parker Drilling Rig #114': {'address': '3rd Street (Historic Route 66) and Madison\nElk City, OK 73644', 'description': "At 181 feet (17 stories), Parker Drilling Rig #114 is one of the world's tallest oil rigs. The rig was originally built in the 1960's to drill shafts for underground nuclear testing and later was used for discovering oil.", 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/c3766ae78c1ce5e808630e2f7423c63c_.jpeg'}, 'Flamingo Inn': {'address': '2000 W 3rd St\nElk City, OK 73644', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/9ba05c6fc84c4d7005d3da926b4a1146_.jpeg'}, 'National Rt 66 Museum': {'address': '320 W 3rd St\nElk City, OK 73644', 'description': 'ROUTE 66 PASSPORT STAMP PLACE - World’s largest Route 66 Sign - 4 Museums in one (National Route 66 & Transportation, Farm & Ranch, Old Town, and Route 66 Blacksmith) - One of the largest Rt. 66 attractions in Oklahoma - 14 ft. tall Kachina Doll greets visitor at museum entrance', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/a31b3d9073ef8ffe5dd980de108c98a6_.jpeg'}, 'Great Western Cattle Trail': {'address': '1513 W 2nd St\nElk City, OK 73644', 'description': 'The Farm and Ranch Museum is sponsored by the Western Oklahoma Historical Society. The museum features a vast collection of tools, tractors and farm equipment that were used in early farm and ranch life. A Blacksmith shop and windmill collection are among the exhibits depicting pioneer farmers and ranchers and their contributions to this nation.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/07aa131fbbda5f6afefb3f1ca13fba64_.jpeg'}, 'Timber Creek Bridge': {'address': 'Old Route 66\nSayre, OK 73662', 'description': 'A through Truss Bridge over Timber Creek built to carry Route 66 traffic in 1926', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/fe3651e233b4b2524971f861f8122ce4_.jpeg'}, 'Western Motel': {'address': '315 N E Hwy 66\nSayre, OK 73662', 'description': 'NULL', 'image': 'https://www.historic66.com/templates/data/images/sharepoi.svg'}, 'Old Alignment Rte 66': {'address': 'Historic Route 66\nErick, OK 73645', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/b6690cf56370e9278eb14501af30ce65_.jpeg'}, 'Windmill Lane': {'address': 'I-40BL\nErick, OK 73645', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/6c7d9843f01e9e93ebeed77d3a32f225_.jpeg'}, 'Roger Miller Museum': {'address': '101 S Sheb Wooley Ave\nErick, OK 73645', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/9020bf0e8cd9c8f71e0d21d4734521bd_.jpeg'}, 'Elm Motel Sign': {'address': '721 Broadway\nErick, OK 73645', 'description': 'NULL', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/90f66f3272140a717f00d9dee20fe042_.jpeg'}, 'Sandhill Curiosity Shop': {'address': '201 S Sheb Wooley Ave\nErick, OK 73645', 'description': 'Located in downtown Erick, this unusual shop offers an experience and atmosphere not soon forgotten. Self-described "redneck hoarder" Harley Russell performs live, spontaneously and interactively for all those who visit the shop. Guests will find turn-of-the-century oak furniture, a variety of artwork, vintage musical instruments and a variety of objects both curious and fascinating. The Sandhills Curiosity Shop, known as the oldest brick building in Erick, can accommodate up to 75 people. Visitors can catch Harley on various days performing for singles, couples, families, tour bus groups and more. Refreshments are free, along with plenty of photo-ops. The Sandhills Curiosity Shop is considered one of the most unique roadside establishments along Route 66.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/45dc38f4697af32f696bb30cda11ece6_.jpeg'}, 'One Room Jail': {'address': 'Main Ave\nTexola, OK 73668', 'description': 'Built in 1910 the single room jail features an iron barred door at the entrance. The interior also includes an iron barred window and displays about notorious outlaws. Outside, there is a randomly placed tombstone-style memorial paying homage to the graduating high school class of 1938.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/7f8cfdc613307d5f5a1e5f14c37ff44e_.jpeg'}, 'Tumbleweed Bar & Grill': {'address': '16726 Rt 66 S\nTexola, OK 73668', 'description': 'A true delight, this charming restaurant with superb pies and wonderful gift shop is a gem.', 'image': 'https://www.historic66.com/wp-content/uploads/2020/12/8253708e50bd56e96c178778e0d74b0d_.jpeg'}}

In [3]:
'''filename = 'historic66.json'
with open(filename, 'w', encoding='utf-8') as f:
    places = json.load(jayson, f)
print(places)'''

NameError: ignored

In [10]:
key =   # leaving key out of repo for 

In [12]:
Places = dict()
periods = dict()
for Place, details in places.items():
    address = detais['address']
    Description = details['description']
    Image = details['image']

    append = Place.replace(' ', '%20') + '%20' + address.replace(' ', '%20').replace('\n', '%20')
    print(append)
    pid = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=' + append + '&inputtype=textquery&key=' + key)
    print(type(pid))
    print(pid.json())
    # for pd in pid.json()['candidates']:
    Place_id = pid.json().get('place_id')
    p =  requests.get('https://maps.googleapis.com/maps/api/place/details/json?place_id=' + Place_id + '&key=' + key)
    result = p.json()['result']
    # print(place)

    USE GET? OUTPUT DEFAULT ON NO VALUE...


    Address = result.get('adr_address', 'NULL')  #regex on street-address span
    City = result.get('adr_address', 'NULL')  #  regex on locality span
    State = result.get('adr_address', 'NULL')  #  regex on region span
    Zip = result.get('adr_address', 'NULL')  #  regex on postal-code span

    Lat = result.get('geometry', 'NULL').get('location', 'NULL').get('lat', 'NULL')
    Long = result.get('geometry', 'NULL').get('location', 'NULL').get('long', 'NULL')
    Email = 'NULL'
    Phone = result.get('formatted_phone_number', 'NULL')
    BookingUrl = result.get('website', 'NULL')
    URL = result.get('url', 'NULL')
    Periods = periods.copy()
    periods_response = result.get('opening_hours').get('periods')
    for k, v in 
    ''' open/close timings'''
    SunOpen = Periods.get('SunOpen')
    SunClose
    MonOpen
    MonClose
    TueOpen
    TueClose
    WedOpen
    print('-------------------')
    Places.update({placeid: [str(2), 'NULL', Place, Description, Address, City, State, Zip, Lat, Long, Email, Phone, BookingUrl, Image, 'NULL', URL, Placeid, ]})

    RoadTripStop
    CategoryID (Put a placeholder here. Someone will assign these manually later)
    RoadTripStopName
    RoadTripStopDescription
    RoadTripStopAddressLine1
    RoadTripStopCity
    RoadTripStopState
    RoadTripStopZip
    RoadTripStopLat
    RoadTripStopLong
    RoadTripStopEmail
    RoadTripStopPhone
    TicketBookingURL
    Image - this is the main image that will display for the location
    EventDateTime - this will be null for entries with RoadTripStopTypeID=2(Location)
    RoadTripStopURL
    PlaceID

    #j.get('candidates')[0]['place_id']
    break

Picher%20–%20Ghost%20Town%20Picher,%20OK%2074360
<class 'requests.models.Response'>
{'candidates': [{'place_id': 'ChIJ5dEhIAsRyIcRmlKi8ZIsd9s'}, {'place_id': 'ChIJ1QeuV3MRyIcRjdKGpIna-z8'}], 'status': 'OK'}
{'html_attributions': [], 'result': {'address_components': [{'long_name': 'Picher', 'short_name': 'Picher', 'types': ['locality', 'political']}, {'long_name': 'Ottawa County', 'short_name': 'Ottawa County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Oklahoma', 'short_name': 'OK', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '74360', 'short_name': '74360', 'types': ['postal_code']}], 'adr_address': '<span class="locality">Picher</span>, <span class="region">OK</span> <span class="postal-code">74360</span>, <span class="country-name">USA</span>', 'formatted_address': 'Picher, OK 74360, USA', 'geometry': {'location': {'lat': 36.9870116, 'lng': -94.

In [ ]:
s = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Blue%20Whale%20Oklahoma&inputtype=textquery&key=' + key)
print(type(s))
s.text


<class 'requests.models.Response'>


'{\n   "candidates" : [\n      {\n         "place_id" : "ChIJHUcdr_X1tocR4qFQGaqLAW8"\n      }\n   ],\n   "status" : "OK"\n}\n'

In [ ]:
j = s.json()
print(type(j))
print(j.get('candidates')[0]['place_id'])
placeid = j.get('candidates')[0]['place_id']

<class 'dict'>
ChIJHUcdr_X1tocR4qFQGaqLAW8


In [ ]:
s = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Hi-Way%20Cafe%20Oklahoma&inputtype=textquery&key=' + key)
print(type(s))
s.text
j = s.json()
print(type(j))
print(j.get('candidates')[0]['place_id'])
placeid = j.get('candidates')[0]['place_id']

<class 'requests.models.Response'>
<class 'dict'>
ChIJ0ePR7rrPt4cRnUfOCj1avVc


In [ ]:
default = dict()
for i in range(7)
    default.update({i: {'open': 'NULL', 'close': 'NULL'}})

oh = {'opening_hours': {'open_now': False,
   'periods': [{'close': {'day': 0, 'time': '1400'},
     'open': {'day': 0, 'time': '0700'}},
    {'close': {'day': 2, 'time': '1400'}, 'open': {'day': 2, 'time': '0800'}},
    {'close': {'day': 3, 'time': '1400'}, 'open': {'day': 3, 'time': '0800'}},
    {'close': {'day': 4, 'time': '1400'}, 'open': {'day': 4, 'time': '0800'}},
    {'close': {'day': 5, 'time': '1400'}, 'open': {'day': 5, 'time': '0800'}},
    {'close': {'day': 6, 'time': '2000'}, 'open': {'day': 6, 'time': '0800'}}]}

for i in oh['opening_hours']['periods']:
    if i == i['close']['day']
    default[i]['close'] = i['close']['day']
    i['open']

In [ ]:
p =  requests.get('https://maps.googleapis.com/maps/api/place/details/json?place_id=' + placeid + '&key=' + key)
place = p.json()
place

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '437918',
    'short_name': '437918',
    'types': ['street_number']},
   {'long_name': 'U.S. 60', 'short_name': 'US-60', 'types': ['route']},
   {'long_name': 'Vinita',
    'short_name': 'Vinita',
    'types': ['locality', 'political']},
   {'long_name': 'Craig County',
    'short_name': 'Craig County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Oklahoma',
    'short_name': 'OK',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '74301', 'short_name': '74301', 'types': ['postal_code']},
   {'long_name': '7644',
    'short_name': '7644',
    'types': ['postal_code_suffix']}],
  'adr_address': '<span class="street-address">437918 US-60</span>, <span class="locality">Vinita</span>, <span class="region">OK</span> <span class="postal-code">74301-7644</span>, 